# resources

> CDN resources and headers for daisyUI and Tailwind CSS

In [ ]:
#| default_exp core.resources

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import Link, Script, Style
from typing import List, Optional, Union, Literal
from pathlib import Path

## Core CDN Resources

The library provides pre-configured CDN headers for daisyUI v5 and Tailwind CSS v4:

In [ ]:
#| export
# Core daisyUI and Tailwind CSS CDN resources
DAISYUI_CDN = "https://cdn.jsdelivr.net/npm/daisyui@5"
DAISYUI_THEMES_CDN = "https://cdn.jsdelivr.net/npm/daisyui@5/themes.css"
DAISYUI_COLOR_PROPERTIES = "https://cdn.jsdelivr.net/npm/daisyui@5/colors/properties.css"
DAISYUI_COLOR_PROPERTIES_EXT = "https://cdn.jsdelivr.net/npm/daisyui@5/colors/properties-extended.css"
TAILWIND_CDN = "https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4"

In [ ]:
#| export
def get_daisyui_headers(
    include_themes: bool = True  # Include the daisyUI themes CSS file
) -> List[Union[Link, Script]]:  # List of Link and Script elements for daisyUI and Tailwind CSS
    """Get the standard daisyUI and Tailwind CSS CDN headers."""
    headers = [
        Link(rel="stylesheet", href=DAISYUI_CDN, type="text/css"),
        Link(rel="stylesheet", href=DAISYUI_COLOR_PROPERTIES, type="text/css"),
        Link(rel="stylesheet", href=DAISYUI_COLOR_PROPERTIES_EXT, type="text/css"),
        Script(src=TAILWIND_CDN)
    ]
    
    if include_themes:
        headers.insert(1, Link(rel="stylesheet", href=DAISYUI_THEMES_CDN, type="text/css"))
    
    return headers

In [ ]:
get_daisyui_headers()

[link((),{'rel': 'stylesheet', 'href': 'https://cdn.jsdelivr.net/npm/daisyui@5', 'type': 'text/css'}),
 link((),{'rel': 'stylesheet', 'href': 'https://cdn.jsdelivr.net/npm/daisyui@5/themes.css', 'type': 'text/css'}),
 link((),{'rel': 'stylesheet', 'href': 'https://cdn.jsdelivr.net/npm/daisyui@5/colors/properties.css', 'type': 'text/css'}),
 link((),{'rel': 'stylesheet', 'href': 'https://cdn.jsdelivr.net/npm/daisyui@5/colors/properties-extended.css', 'type': 'text/css'}),
 script(('',),{'src': 'https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4'})]

Example usage:

In [ ]:
# Get all headers including themes
headers = get_daisyui_headers()
print(f"Number of headers: {len(headers)}")
for h in headers:
    print(h)

Number of headers: 5
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5" type="text/css">
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5/themes.css" type="text/css">
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5/colors/properties.css" type="text/css">
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5/colors/properties-extended.css" type="text/css">
<script src="https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4"></script>


In [ ]:
# Get headers without themes (for custom theme usage)
headers_no_themes = get_daisyui_headers(include_themes=False)
print(f"Number of headers without themes: {len(headers_no_themes)}")
for h in headers_no_themes:
    print(h)

Number of headers without themes: 4
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5" type="text/css">
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5/colors/properties.css" type="text/css">
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5/colors/properties-extended.css" type="text/css">
<script src="https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4"></script>


## Custom Resources

For adding custom CSS files, JavaScript libraries, or local theme files:

In [ ]:
#| export
def create_css_link(
    href: str,  # URL or path to CSS file
    media: Optional[str] = None,  # Media query (e.g., "screen", "print")
    crossorigin: Optional[Literal["anonymous", "use-credentials"]] = None
) -> Link:  # Link element for CSS stylesheet
    """Create a CSS link element with optional attributes."""
    attrs = {"rel": "stylesheet", "href": href, "type": "text/css"}
    
    if media:
        attrs["media"] = media
    if crossorigin:
        attrs["crossorigin"] = crossorigin
        
    return Link(**attrs)

In [ ]:
#| export
def create_js_script(
    src: str,  # URL or path to JavaScript file
    async_: bool = False,  # Load script asynchronously
    defer: bool = False,  # Defer script execution
    module: bool = False,  # ES6 module
    crossorigin: Optional[Literal["anonymous", "use-credentials"]] = None
) -> Script:  # Script element for JavaScript file
    """Create a JavaScript script element with optional attributes."""
    attrs = {"src": src}
    
    if async_:
        attrs["async"] = ""
    if defer:
        attrs["defer"] = ""
    if module:
        attrs["type"] = "module"
    if crossorigin:
        attrs["crossorigin"] = crossorigin
        
    return Script(**attrs)

## Combined Header Builder

A comprehensive function to build all headers with custom resources:

In [ ]:
#| export
def build_headers(
    include_themes: bool = True,  # Include daisyUI themes
    custom_css: Optional[List[Union[str, Link]]] = None,  # Additional CSS files
    custom_js: Optional[List[Union[str, Script]]] = None,  # Additional JS files
    custom_theme_css: Optional[str] = None,  # Custom theme CSS as a string
    custom_theme_paths: Optional[List[Union[str, Path]]] = None  # List of paths to custom theme CSS files
) -> List[Union[Link, Script, Style]]:  # List of Link, Script, and Style elements for complete app headers
    """
    Build a complete set of headers for a FastHTML app with daisyUI and Tailwind.
    
    The order of headers is:
    1. daisyUI CSS
    2. daisyUI themes CSS (if included)
    3. Custom theme CSS (if provided as string)
    4. Custom theme CSS files (if provided as Path objects)
    5. Custom CSS files
    6. Tailwind CSS JavaScript
    7. Custom JavaScript files
    """
    headers = []
    
    # Core daisyUI CSS
    headers.append(Link(rel="stylesheet", href=DAISYUI_CDN, type="text/css"))
    headers.append(Link(rel="stylesheet", href=DAISYUI_COLOR_PROPERTIES_EXT, type="text/css"))
    
    # daisyUI themes
    if include_themes:
        headers.append(Link(rel="stylesheet", href=DAISYUI_THEMES_CDN, type="text/css"))
    
    # Custom theme CSS as inline style
    if custom_theme_css:
        headers.append(Style(custom_theme_css))
    
    # Custom theme paths - load CSS content as Style elements
    if custom_theme_paths:
        for theme_path in custom_theme_paths:
            path = Path(theme_path)
            if path.exists() and path.is_file():
                with open(path, 'r', encoding='utf-8') as f:
                    headers.append(Style(f.read()))
    
    # Custom CSS files
    if custom_css:
        for css in custom_css:
            if isinstance(css, str):
                headers.append(create_css_link(css))
            else:
                headers.append(css)
    
    # Tailwind CSS (must come after daisyUI for proper initialization)
    headers.append(Script(src=TAILWIND_CDN))
    
    # Custom JavaScript files
    if custom_js:
        for js in custom_js:
            if isinstance(js, str):
                headers.append(create_js_script(js))
            else:
                headers.append(js)
    
    return headers

Example with custom resources:

In [ ]:
# Build headers with custom resources
from nbdev.config import get_config
cfg = get_config()
project_dir = cfg.config_path

custom_headers = build_headers(
    include_themes=True,
    custom_css=[
        "/static/custom.css",
        "https://cdn.example.com/fonts.css"
    ],
    custom_js=[
        create_js_script("/static/app.js", defer=True),
        "https://cdn.example.com/analytics.js"
    ],
    custom_theme_paths=[project_dir / "css" / "custom_light_theme.css"]
)

print(f"Total headers: {len(custom_headers)}")
for i, h in enumerate(custom_headers):
    print(f"{i+1}. {h}")

Total headers: 9
1. <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5" type="text/css">
2. <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5/colors/properties-extended.css" type="text/css">
3. <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5/themes.css" type="text/css">
4. <style>:root:has(input.theme-controller[value=custom_light_theme]:checked),[data-theme="custom_light_theme"] {
  color-scheme: light;
  --color-base-100: oklch(98% 0.005 220);
  --color-base-200: oklch(96% 0.008 215);
  --color-base-300: oklch(92% 0.012 210);
  --color-base-content: oklch(18% 0.015 230);
  --color-primary: oklch(55% 0.18 260);
  --color-primary-content: oklch(98% 0.005 260);
  --color-secondary: oklch(45% 0.12 340);
  --color-secondary-content: oklch(98% 0.005 340);
  --color-accent: oklch(65% 0.15 180);
  --color-accent-content: oklch(15% 0.01 180);
  --color-neutral: oklch(25% 0.01 240);
  --color-neutral-content: oklch(95% 0.005 240);
  --colo

### Creating CSS Links with Media and CORS

The `create_css_link()` function supports media queries and CORS settings:

In [ ]:
# Basic CSS link
basic_css = create_css_link("/static/styles.css")
print("Basic CSS link:")
print(basic_css)

# CSS link with media query for print styles
print_css = create_css_link(
    href="/static/print.css",
    media="print"
)
print("\nPrint-only CSS link:")
print(print_css)

# CSS link for dark mode with media query
dark_mode_css = create_css_link(
    href="/static/dark-theme.css",
    media="(prefers-color-scheme: dark)"
)
print("\nDark mode CSS link:")
print(dark_mode_css)

# External CSS with CORS enabled (for fonts from CDN)
font_css = create_css_link(
    href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap",
    crossorigin="anonymous"
)
print("\nExternal font CSS with CORS:")
print(font_css)

# CSS for large screens only
desktop_css = create_css_link(
    href="/static/desktop.css",
    media="screen and (min-width: 1024px)"
)
print("\nDesktop-only CSS link:")
print(desktop_css)

Basic CSS link:
<link rel="stylesheet" href="/static/styles.css" type="text/css">

Print-only CSS link:
<link rel="stylesheet" href="/static/print.css" type="text/css" media="print">

Dark mode CSS link:
<link rel="stylesheet" href="/static/dark-theme.css" type="text/css" media="(prefers-color-scheme: dark)">

External font CSS with CORS:
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&amp;display=swap" type="text/css" crossorigin="anonymous">

Desktop-only CSS link:
<link rel="stylesheet" href="/static/desktop.css" type="text/css" media="screen and (min-width: 1024px)">


### Creating JavaScript Scripts with Attributes

Similarly, `create_js_script()` supports various loading strategies:

In [ ]:
# Example: Building a complete set of headers with media queries and CORS
complete_headers = build_headers(
    include_themes=True,
    custom_css=[
        create_css_link("/static/base.css"),
        create_css_link("/static/print.css", media="print"),
        create_css_link(
            "https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;700",
            crossorigin="anonymous"
        ),
        create_css_link(
            "/static/mobile.css", 
            media="screen and (max-width: 768px)"
        )
    ],
    custom_js=[
        create_js_script("/static/app.js", defer=True),
        create_js_script("/static/analytics.js", async_=True),
        create_js_script(
            "https://cdn.jsdelivr.net/npm/alpinejs@3.x.x/dist/cdn.min.js",
            defer=True,
            crossorigin="anonymous"
        )
    ]
)

print(f"Complete headers with media queries and CORS ({len(complete_headers)} total):\n")
for i, header in enumerate(complete_headers, 1):
    print(f"{i}. {header}")

Complete headers with media queries and CORS (11 total):

1. <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5" type="text/css">
2. <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5/colors/properties-extended.css" type="text/css">
3. <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/daisyui@5/themes.css" type="text/css">
4. <link rel="stylesheet" href="/static/base.css" type="text/css">
5. <link rel="stylesheet" href="/static/print.css" type="text/css" media="print">
6. <link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;700" type="text/css" crossorigin="anonymous">
7. <link rel="stylesheet" href="/static/mobile.css" type="text/css" media="screen and (max-width: 768px)">
8. <script src="https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4"></script>
9. <script src="/static/app.js"></script>
10. <script src="/static/analytics.js"></script>
11. <script src="https://cdn.jsdelivr.net/npm/alpinejs@3.x.x/dist/cd

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()